In [10]:
from pathlib import Path

import earthaccess
import h5py

In [2]:
auth = earthaccess.login()

In [42]:
results = earthaccess.search_data(short_name="ATL07",
                                  temporal=("2024-01-01","2024-01-02"),
                                  cloud_hosted=True,
                                  count=1)

len(results)

1

In [43]:
f = h5py.File(earthaccess.open([results[0]])[0])

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
variables = {}

def get_attributes(obj):
    attnames = [
        "coordinates",
        "description",
        "long_name",
        "source",
        "standard_name",
        "units",
    ]
    return {attnm: obj.attrs.get(attnm) for attnm in attnames} 
    
def make_entry(path, obj):
    if isinstance(obj, h5py.Dataset):
        variables[path] = {
            "name": Path(path).name,
            "attrs": get_attributes(obj),
        }
        
f.visititems(make_entry)

In [45]:
len(variables)

712

In [47]:
from pprint import pprint

pprint(variables, indent=4)

{   'ancillary_data/atlas_sdp_gps_epoch': {   'attrs': {   'coordinates': None,
                                                           'description': b'Number of GPS seconds between the GPS epoch (1980-01-06T00:00:00.000000Z UTC) and the ATLAS Standard Data Product (SDP) epoch (2018-01-01:T00.00.00.000000 UTC). Add this value to delta time parameters to compute full gps_seconds (relative to the GPS epoch) for each data point.',
                                                           'long_name': b'ATLAS Epoch Offset',
                                                           'source': b'Operations',
                                                           'standard_name': None,
                                                           'units': b'seconds since 1980-01-06T00:00:00.000000Z'},
                                              'name': 'atlas_sdp_gps_epoch'},
    'ancillary_data/coarse_surface_finding/bin_c': {   'attrs': {   'coordinates': None,
                     

In [37]:
results[0].data_links()

['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2023/12/31/ATL07-01_20231231233322_02012201_006_01.h5']

In [33]:
import rasterio

In [40]:
with rasterio.open(earthaccess.open([results[0]])[0]) as src:
    print("Driver:", src.driver)
    print("Width:", src.width)
    print("Height:", src.height)
    print("Bands:", src.count)
    print("CRS:", src.crs)
    print("Bounds:", src.bounds)
    print("Transform:", src.transform)

    for subdataset in src.subdatasets:
        print("Subdatasets", subdataset)

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Driver: HDF5
Width: 512
Height: 512
Bands: 0
CRS: None
Bounds: BoundingBox(left=0.0, bottom=512.0, right=512.0, top=0.0)
Transform: | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|
Subdatasets HDF5:/vsipythonfilelike/57d0135f-5979-4022-829d-34e233b40070/57d0135f-5979-4022-829d-34e233b40070://ancillary_data/atlas_engineering/ph_uncorrelated_error
Subdatasets HDF5:/vsipythonfilelike/57d0135f-5979-4022-829d-34e233b40070/57d0135f-5979-4022-829d-34e233b40070://ancillary_data/atlas_engineering/transmit/tx_pulse_width_upper
Subdatasets HDF5:/vsipythonfilelike/57d0135f-5979-4022-829d-34e233b40070/57d0135f-5979-4022-829d-34e233b40070://quality_assessment/gt3l/qa_perc_signal_conf_ph_low
Subdatasets HDF5:/vsipythonfilelike/57d0135f-5979-4022-829d-34e233b40070/57d0135f-5979-4022-829d-34e233b40070://quality_assessment/gt3l/qa_perc_signal_conf_ph_med
Subdatasets HDF5:/vsipythonfilelike/57d0135f-5979-4022-829d-34e233b40070/57d0135f-5979-4022-829d-34e233b40070://quality_assessment/gt3l/qa_p

SystemError: <built-in function exc_info> returned a result with an exception set

## Querying Quishings Big List

In [18]:
import pandas as pd

url = 'https://github.com/opengeos/NASA-Earth-Data/raw/main/nasa_earth_data.tsv'
df = pd.read_csv(url, sep='\t')
df.head()

,ShortName,EntryTitle,DOI,concept-id,provider-id,s3-links,bbox-crs,bbox,horizontal_res,start-time,end-time,Creator,Publisher,ReleaseDate,ReleasePlace,Version,Linkage
0,2009oct_Chesapeake,Chesapeake Bay measurements during October 2009,10.5067/SeaBASS/2009OCT_CHESAPEAKE/DATA001,C1633360083-OB_DAAC,OB_DAAC,NaN,CARTESIAN,"[-180.0, -90.0, 180.0, 90.0]",NaN,2009-10-12T00:00:02Z,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.5067/SeaBASS/2009OCT_CHESAP...
1,2dimpacts,Two-Dimensional Video Disdrometer (2DVD) IMPAC...,10.5067/IMPACTS/2DVD/DATA101,C1995564612-GHRC_DAAC,GHRC_DAAC,['ghrcw-protected/2dimpacts__1/'],CARTESIAN,"[-75.4912, 37.9194, -75.4462, 37.9543]",NaN,2020-01-15T01:37:06.000Z,2020-02-28T20:41:44.000Z,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.5067/IMPACTS/2DVD/DATA101
2,AAOT,Acqua Alta Oceanographic Tower (AAOT),10.5067/SeaBASS/AAOT/DATA001,C1633360084-OB_DAAC,OB_DAAC,NaN,CARTESIAN,"[-180.0, -90.0, 180.0, 90.0]",NaN,1999-08-03T00:00:02Z,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.5067/SeaBASS/AAOT/DATA001
3,AA_L2A,AMSR/ADEOS-II L2A Global Swath Spatially-Resam...,10.5067/ADEOS-II/AMSR/AA_L2A.001,C1241435536-NSIDC_ECS,NSIDC_ECS,NaN,CARTESIAN,"[-180.0, -90.0, 180.0, 90.0]",NaN,2003-01-18T00:00:00.000Z,2003-10-24T23:59:59.999Z,NaN,NASA National Snow and Ice Data Center Distrib...,NaN,NaN,1,https://doi.org/10.5067/ADEOS-II/AMSR/AA_L2A.001
4,ABI_G16-STAR-L2P-v2.70,GHRSST NOAA/STAR GOES-16 ABI L2P America Regio...,10.5067/GHG16-2PO27,C2036877465-POCLOUD,POCLOUD,['podaac-ops-cumulus-protected/ABI_G16-STAR-L2...,CARTESIAN,"[-135.0, -59.0, -15.0, 59.0]","[{'XDimension': 2000.0, 'YDimension': 2000.0, ...",2017-12-15T00:00:00.000Z,NaN,NOAA/NESDIS/STAR,PO.DAAC,2019-05-15T00:00:00.000Z,"Camp Springs, MD (USA)",2.70,https://doi.org/10.5067/GHG16-2PO27


In [24]:
df.describe()

,ShortName,EntryTitle,DOI,concept-id,provider-id,s3-links,bbox-crs,bbox,horizontal_res,start-time,end-time,Creator,Publisher,ReleaseDate,ReleasePlace,Version,Linkage
count,8544,8544,7895,8544,8544,5174,7197,7197,1562,8466,6065,3353,3856,4441,3243,3407,7895
unique,8067,8505,7812,8544,24,5134,2,2101,197,3063,2763,609,79,1086,80,154,7812
top,GloSSAC,GLAS/ICESat L2 Global Planetary Boundary Layer...,10.5270/S5P-kb39wni,C2813517035-ORNL_CLOUD,ORNL_CLOUD,['s3://ob-cumulus-prod-public/'],CARTESIAN,"[-180.0, -90.0, 180.0, 90.0]","[{'Unit': 'Decimal Degrees', 'XDimension': 1.0...",2004-08-02T00:00:00.000Z,2015-06-07T12:45:21.000Z,NASA Aquarius project,Goddard Earth Sciences Data and Information Se...,2017-12-07T00:00:00.000Z,"Greenbelt, MD, USA",1,https://doi.org/10.5270/S5P-kb39wni
freq,6,2,8,1,1862,26,6873,1656,180,145,118,175,1698,175,1354,801,8
